![](images/two-stream-attention.PNG)

为了实现 Permutation 加上 AR 预测过程，首先我们会发现，打乱顺序后位置信息非常重要，同时对每个位置来说，需要预测的是内容信息（对应位置的词），于是输入就不能包含内容信息，不然模型学不到东西，只需要直接从输入复制到输出就好了。

于是这里就造成了位置信息与内容信息的割裂，因此在 BERT 这样的位置信息加内容信息输入 Self-Attention （自注意力） 的流（Stream）之外，作者还增加了另一个只有位置信息作为 Self-Attention 中 Query 输入的流。文中将前者称为 Content Stream，而后者称为 Query Stream。

这样就能利用 Query Stream 在对需要预测位置进行预测的同时，又不会泄露当前位置的内容信息。具体操作就是用两组隐状态 g和 h 。其中  g  只有位置信息，作为 Self-Attention 里的 Q 。h 包含内容信息，则作为 K和V 。具体表示如上图图所示：

第一点，最下面一层蓝色的 Content Stream 的输入是$e(x_i)$，这个很好懂就是$x$ 对应的词向量 (Embedding)，不同词对应不同向量，但看旁边绿色的 Query Stream，就会觉得很奇怪，为什么都是一样的 $w$？这个和Relative Positional Encoding 有关。
第二点，Query stream attention图中为了便于说明，只将当前位置之外的 h
作为K  和 V，但实际上实现中应该是所有时序上的 h 都作为 K 和 V，最后再交给上图中的 Query stream 的 Attention Mask 来完成位置的遮盖。

# XLNet 模型
原始论文：   
XLNet: Generalized Autoregressive Pretraining for Language Understanding
https://arxiv.org/abs/1906.08237    
源代码：https://github.com/zihangdai/xlnet               
中文：https://github.com/ymcui/Chinese-XLNet

## 预训练语言模型
自回归语言模型，`AR：Autoregressive Language Modeling`
   - 依据前面（或后面）出现的`tokens`来预测当前时刻的`token`，代表有 `ELMO`， `GPT`等。
   - 它只能利用单向语义而不能同时利用上下文信息。`ELMO` 通过双向都做`AR` 模型，然后进行拼接，但从结果来看，效果并不是太好。
   - 对生成模型友好，天然符合生成式任务的生成过程。这也是为什么 `GPT` 能够编故事的原因
    
    
自编码语言模型，`AE：Autoencoding Language Modeling`
   - 通过上下文信息来预测被`mask`的`token`，代表有 `BERT` , `Word2Vec(CBOW)`  。
   - 由于训练中采用了 `[MASK]` 标记，导致预训练与微调阶段不一致的问题。BERT独立性假设问题，即没有对被遮掩（`Mask`）的 `token` 之间的关系进行学习。此外对于生成式问题， AE 模型也显得捉襟见肘。
   - 能够很好的编码上下文语义信息（即考虑句子的双向信息）， 在自然语言理解相关的下游任务上表现突出。

## XLNet 的改进
BERT 问题：
- BERT 在预训练中使用 [MASK]，而在调优的真实数据中并不存在，两个阶段存在使用模式不一致的情形，这可能会带来一定的性能损。
- [MASK] 的另一个缺点是它假设这些被Mask掉的单词之间没有任何关系，是条件独立的，而有时候这些单词之间是有关系的，XLNet则考虑了这种关系

XLNet 改进：
- 以 AR 的形式的从左向右的输入，依次进行预测，而不是 bert 的 AE 形式；同时在预测时，不仅使用到可上文信息，还获取了下文的信息
- 如何在序列当前单词前面添加进下文的信息？

### 排列语言建模
`XLNet`结合AR和AE模型的优点，提出了在预训练阶段使用的 `Permutation Language Model（PLM）`：
- 将长度为$T$的序列$\{x_1,x_2,x_3,x_4\}$，重新排列，共有$T!$种排列 --> <span class="burk">(如何实现？？)</span>，如下图所示
    - 如序列$\{x_1,x_2,x_3,x_4\}$，重排成$\{x_1,x_4,x_3,x_2\}$
    
<img src="../images/xlnet.jpg" width="100%">    
- 从中随机选择一种排列，然后将末尾一定量的词给遮掩掉 --> <span class="burk">(如何抽样，如何遮掩)</span>，然后按照`AR`的方式依次预测被遮掩掉的词
> 如上图所示：要预测位置 3 处的单词 $x_3$ ,为了获取下文 $x_4$ 的信息：
> - 从重排列序列中选择 `2->4->3->1` 序列，则预测  $x_3$ 时不仅利用了上文 $x_1$ 的信息，还利用了下文 $x_4$ 的信息   
> - 而 `3->2->1->4` 则只能利用初始信息
> - 排列`1->4->2->3`，则利用了 $x_1, x_4, x_2$的信息
> - 排列`4->3->1->2`，则只能利用了 $x_4$的信息
    
- 形式上从左到右依次预测，实际已经获取了上下文的信息

  -------------
  
- 原理如上所述，但实际实现时不可能把输入序列的单词排序组合，再抽样作为输入：因为调优阶段是不可能随机排列组合输入序列的
- 使用了 用“双流自注意力模型” 实现

### 双流自注意力机制

具体的实现方式如下图所示，通过对 Attention Mask 进行操作:
<img src="../images/two-stream-attention.jpg" width="100%">
- 图 (a) 所示为标准的自注意力机制，对整个序列进行注意力操作；
- 图 (b) 为 Query stream attention，对自身以外的整个序列进行注意力操作
- 图 (c) 为双流自注意力机制：
    - 对于序列 `3->2->4->1`，如图中右下部分红色所示，每一行表示 $x_i$ 可以看见序列中哪些单词
    - 计算 $x_1$ 需对 $x_3, x_2, x_4$ 进行注意力操作
    - 计算 $x_2$ 需对 $x_3$ 进行注意力操作
    - 计算 $x_4$ 需对 $x_3, x_2$ 进行注意力操作
    
- 因为还需要计算 重排序列，XLNet 的计算量更大了，因此简化为只预测后 1/k 个标记    

# 基于 XLNet 的文本分类

In [6]:
import torch
from transformers import XLNetTokenizer, XLNetModel

model_path = "../../H/models/huggingface/xlnet_chinese_base/"
tokenizer = XLNetTokenizer.from_pretrained(model_path)
model = XLNetModel.from_pretrained(model_path)

In [7]:
model

XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, b

In [8]:
tokenizer.encode("深度学习很强大！")

[19, 5755, 1689, 711, 16772, 1039, 4, 3]

In [9]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("深度学习很强大！"))

['▁', '深度', '学习', '很', '强大', '!', '<sep>', '<cls>']

In [11]:
tokenizer.vocab_size

32000

In [12]:
tokenizer.all_special_ids

[5, 2, 6, 0, 7, 8, 1, 3, 4]

In [15]:
for i in range(tokenizer.vocab_size):
    word = tokenizer.convert_ids_to_tokens([i])
    print(f"{i}: {word[0]}")

0: <unk>
1: <s>
2: </s>
3: <cls>
4: <sep>
5: <pad>
6: <mask>
7: <eod>
8: <eop>
9: .
10: (
11: )
12: "
13: -
14: –
15: £
16: €
17: ,
18: 。
19: ▁
20: 的
21: 、
22: 年
23: 在
24: 和
25: 月
26: 是
27: 人
28: 中
29: 日
30: :
31: 》
32: ·
33: 有
34: 為
35: 了
36: 《
37: 以
38: 「
39: 为
40: 與
41: 1
42: 上
43: 及
44: 大
45: 2
46: 於
47: “
48: ”
49: ;
50: 被
51: 3
52: 而
53: 由
54: 等
55: 」
56: 5
57: 4
58: 之
59: 一
60: 後
61: 他
62: 6
63: 不
64: 到
65: ▁在
66: 与
67: 7
68: 也
69: 8
70: 其
71: 時
72: 于
73: 9
74: 至
75: 或
76: 下
77: 10
78: 地
79: 德
80: 子
81: 所
82: 名
83: 山
84: 新
85: 后
86: 三
87: 多
88: 西
89: 但
90: 12
91: 前
92: 特
93: 並
94: 文
95: 家
96: 小
97: 米
98: 高
99: 市
100: 者
101: 會
102: 斯
103: 王
104: 出
105: 南
106: 11
107: 都
108: 將
109: 科
110: 州
111: 水
112: 道
113: 法
114: 性
115: 时
116: 对
117: 對
118: 天
119: 可
120: 以及
121: 使用
122: 向
123: 里
124: 分
125: 海
126: 并
127: 第
128: 城
129: 成
130: 克
131: 外
132: 因
133: 站
134: 将
135: 路
136: 用
137: 县
138: 拉
139: 金
140: 自
141: /
142: 平方公里
143: 林
144: 区
145: 安
146: 比
147: 包括
148: 部
149: 利
150: s
151: 化
15

5341: 阶
5342: 直播
5343: 1973
5344: 腰
5345: 割
5346: 有許多
5347: 人数
5348: 航班
5349: 巫
5350: 規
5351: 的主
5352: km
5353: 主要的
5354: 港鐵
5355: 測
5356: 年時
5357: ph
5358: 上面
5359: 全长
5360: 省負責管轄
5361: 箭
5362: 棘
5363: 她在
5364: 140
5365: 如同
5366: 1972
5367: 矛盾
5368: 污染
5369: 的地位
5370: 视频
5371: 採取
5372: 我的
5373: 铺
5374: 人生
5375: 男女
5376: 粒
5377: Al
5378: 原則
5379: 有一個
5380: 通訊
5381: 传播
5382: 一間
5383: 瓦尔
5384: 的儿子
5385: 俠
5386: 成立于
5387: SAO
5388: 吐
5389: 传说
5390: 新建
5391: 西亞
5392: 畢
5393: 原理
5394: 电视剧
5395: 殷
5396: 稳定
5397: 四大
5398: 蜂
5399: 装备
5400: 縱
5401: 西南部
5402: 以上的
5403: 銀緯
5404: 筒
5405: 內閣
5406: MV
5407: 农民
5408: 900
5409: 130
5410: ▁1989
5411: 教學
5412: 邀请
5413: 眾多
5414: 滨
5415: 救援
5416: 羅斯
5417: 1974
5418: 同名
5419: 恆
5420: ▁对于
5421: 位于銀經
5422: 研究生
5423: 叫做
5424: 的前
5425: 漫画
5426: ▁除
5427: 運行
5428: 的支持
5429: ▁第二
5430: 装置
5431: 適
5432: ca
5433: 商場
5434: 彌
5435: 卡斯
5436: 劫
5437: 任職
5438: 向量
5439: 結束後
5440: 灭
5441: 开展
5442: 仪式
5443: 被捕
5444: 運用
5445: 親王
5446: 他被
5447: 的能力
5448: 頗
5449: 以便
5450: IC
5

10458: 第一季
10459: 138
10460: 二等
10461: 有利
10462: 千瓦
10463: 遊戲中
10464: 的大部分
10465: 主场
10466: 在當時
10467: 莱昂
10468: 千人
10469: ▁四
10470: ▁它们
10471: 约瑟夫
10472: 典禮
10473: 嚴重的
10474: 級別
10475: 吉爾
10476: 孚
10477: 的經濟
10478: 數碼
10479: 西德
10480: 前來
10481: 發生於
10482: 形態
10483: 移除
10484: 南側
10485: 霜
10486: 侨
10487: 米高
10488: 太宗
10489: 足球运动员
10490: 戚
10491: 缝
10492: 赏
10493: 隊員
10494: 为止
10495: 準決賽
10496: 吏部
10497: 拉特
10498: 学士
10499: 可见
10500: ▁赵
10501: 局局长
10502: org
10503: 上演
10504: 世宗
10505: 選民
10506: ▁学校
10507: 丞相
10508: 主要有
10509: 據說
10510: 戰機
10511: 位列
10512: 主义者
10513: 推理
10514: 雄性
10515: 大部分的
10516: 駛
10517: 大队
10518: Bu
10519: 殺人
10520: 在全球
10521: 编写
10522: 貿
10523: 壹
10524: 政治委员
10525: 慢慢
10526: 沼
10527: 硫酸
10528: 適應
10529: 延伸至
10530: 世界的
10531: 末端
10532: 性和
10533: mp
10534: 莱茵
10535: 的母親
10536: 东南部
10537: 以至
10538: 夜間
10539: 蜘蛛
10540: 障碍
10541: 德州
10542: 巡迴
10543: 違
10544: 蘿
10545: 暂
10546: 是唯一
10547: ding
10548: 臥
10549: 灌木
10550: 可汗
10551: 修訂
10552: 布局
10553: 奇幻
10554: 面临
10555: ▁吴
10

15549: 长时间
15550: 優異
15551: 途徑
15552: 毗鄰
15553: 英皇
15554: 控股有限公司
15555: 主要為
15556: 团长
15557: 一一
15558: 為首的
15559: ual
15560: 15%
15561: 河內
15562: 臨床
15563: 佛像
15564: AG
15565: 佛罗里达州
15566: ▁與此同時
15567: 流星
15568: 傳統上
15569: 京师
15570: 信用
15571: 威尔士
15572: 屏東縣
15573: 总公司
15574: 金朝
15575: 梅爾
15576: 鲁斯
15577: 1865
15578: 个行政村
15579: 省会
15580: 坟
15581: 情景
15582: 幹部
15583: 累计
15584: 作出了
15585: 和社会
15586: 擊中
15587: 翟
15588: 昊
15589: 印度教
15590: 探險隊
15591: 商标
15592: 士官
15593: 司机
15594: 台語
15595: 是南極洲的山峰
15596: 一番
15597: 纠
15598: 对比
15599: 他将
15600: 戰績
15601: 博尔
15602: 常栖息在
15603: 礼拜
15604: 一颗
15605: 行銷
15606: 肽
15607: 工学院
15608: 總數
15609: 苗栗
15610: 整数
15611: 的名義
15612: 两层
15613: 時間內
15614: 个月后
15615: 閻
15616: 布魯
15617: King
15618: 西門
15619: 广义
15620: 法蘭克
15621: 并获得
15622: 已经被
15623: 九江
15624: 擊出
15625: ▁她是
15626: 也因為
15627: lor
15628: 紀元
15629: 濁
15630: 基尔
15631: 戰役中
15632: 一層
15633: ó
15634: 的主力
15635: 監視
15636: 优先
15637: Team
15638: 總統府
15639: Mark
15640: 鬱
15641: 腐败
15642: 財富
15643: 华中
15644:

20571: 壬辰
20572: 莫夫
20573: 栏目
20574: 对外开放
20575: 關懷
20576: 科植物
20577: 馬拉松
20578: 隨着
20579: 多瑙河
20580: 電器
20581: 重創
20582: 誣
20583: 1843
20584: 残留
20585: 富士電視台
20586: 海底隧道
20587: 氏家族
20588: 發展商
20589: 伴隨
20590: 渡邊
20591: 上車人次
20592: CCTV
20593: 引來
20594: Cast
20595: vent
20596: 解救
20597: 元旦
20598: 称帝
20599: 大马士革
20600: 万亩
20601: 源頭
20602: 6°
20603: 膺
20604: 江泽民
20605: 拉脫維亞
20606: 半衰期
20607: 顯示器
20608: 供應商
20609: 探讨
20610: 轉運
20611: 祭拜
20612: CEO
20613: 称其为
20614: 開墾
20615: 所需要的
20616: 中央委员
20617: 保護區
20618: 神圣罗马帝国
20619: 命题
20620: 所羅門
20621: 证券交易所
20622: 注目
20623: 蘋果日報
20624: Bal
20625: 昭宗
20626: 受限
20627: 留意
20628: 破碎
20629: 表決
20630: 超音速
20631: 省略
20632: 喃
20633: Girl
20634: 開封
20635: 绑架
20636: 日本東京都
20637: sky
20638: 硬幣
20639: 合作社
20640: 埔里
20641: 輔音
20642: 少見
20643: 骚乱
20644: 粹
20645: inc
20646: 自製
20647: 不信任
20648: 并将其
20649: 订购
20650: 瞳
20651: 美濃
20652: 本屆賽事
20653: USB
20654: 評分
20655: 党委副书记
20656: 这本书
20657: 层面
20658: 艰
20659: trans
20660: 避難
20661: 首輪
20662: oni
20663: 国营
20664:

25616: 胝
25617: 嘈
25618: 棨
25619: 麿
25620: 鲟
25621: 诘
25622: ว
25623: 黝
25624: 衹
25625: 鹞
25626: 纫
25627: 俪
25628: 炘
25629: 徨
25630: ى
25631: 龑
25632: 噗
25633: 锗
25634: 粧
25635: 燹
25636: 剁
25637: 蒔
25638: 鋯
25639: 甑
25640: 痣
25641: 嶝
25642: 呦
25643: 炕
25644: 鬻
25645: 鋇
25646: 佇
25647: 栒
25648: 咭
25649: 銼
25650: 鱟
25651: 锴
25652: 啰
25653: ท
25654: 侈
25655: 郇
25656: 頫
25657: 怙
25658: 槙
25659: 訌
25660: 鸛
25661: 菩
25662: 剡
25663: 塽
25664: 枞
25665: 対
25666: Γ
25667: 妫
25668: 尙
25669: 饉
25670: 舢
25671: 铆
25672: ∶
25673: 俣
25674: †
25675: 鮄
25676: 鄣
25677: 하
25678: 鍶
25679: 한
25680: 诙
25681: 砻
25682: 枸
25683: 莨
25684: 窰
25685: 谑
25686: 膀
25687: 沌
25688: 睺
25689: 纭
25690: 訃
25691: 濉
25692: 성
25693: 団
25694: स
25695: 瀅
25696: 锵
25697: 薏
25698: 儆
25699: 绷
25700: 가
25701: 俁
25702: ∈
25703: 郾
25704: 鱀
25705: 圮
25706: 狽
25707: Э
25708: 嫚
25709: 壆
25710: 蔼
25711: 湫
25712: 鄖
25713: 擧
25714: 鳐
25715: 鲹
25716: 螨
25717: 縈
25718: 輦
25719: 罘
25720: 垅
25721: 諶
25722: 褔
25723: ύ
25724: 雑
25725: 喰
25726: 嗩
2

30741: 裰
30742: 謦
30743: 妌
30744: 瘺
30745: 綣
30746: 垶
30747: 妸
30748: 尨
30749: 屛
30750: 呲
30751: 揵
30752: 逦
30753: 謏
30754: 魕
30755: 昡
30756: 滟
30757: 靼
30758: 釈
30759: 勧
30760: 奵
30761: 潩
30762: 纟
30763: 薲
30764: 篰
30765: 鹁
30766: 綪
30767: 擥
30768: 軺
30769: 醸
30770: 羼
30771: 頞
30772: 臙
30773: ═
30774: ည
30775: އ
30776: ျ
30777: 필
30778: ྷ
30779: 擞
30780: 澋
30781: 厠
30782: 呴
30783: 骹
30784: ᅥ
30785: 珘
30786: 鄹
30787: 岽
30788: 畵
30789: 鋋
30790: 錝
30791: 룡
30792: 㻑
30793: 閪
30794: 눈
30795: 揃
30796: 璒
30797: 牋
30798: 礜
30799: 缋
30800: 藳
30801: 趆
30802: ぅ
30803: ㆬ
30804: 囎
30805: 諠
30806: 鴜
30807: 茇
30808: ÿ
30809: 愽
30810: 覊
30811: 镔
30812: ܢ
30813: ই
30814: ਸ
30815: ள
30816: ᅳ
30817: 杩
30818: 祃
30819: 雳
30820: থ
30821: ᄐ
30822: 媁
30823: 徤
30824: 怤
30825: 柎
30826: 楶
30827: 瑎
30828: 簏
30829: 鱙
30830: 鶡
30831: উ
30832: ষ
30833: 嫰
30834: 笭
30835: 艎
30836: 蔀
30837: 鐒
30838: ਹ
30839: റ
30840: ත
30841: ᄌ
30842: ἴ
30843: 冫
30844: 擐
30845: 晅
30846: 櫝
30847: 淐
30848: 煶
30849: 甽
30850: 霱
30851: 충
3